In [1]:
import os

In [2]:
%pwd

'd:\\UAE_USED_CAR_DEEP_LEARNING\\UAE_USED_CAR_DEEP_LEARNING_ANALYSIS_PROJECT\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\UAE_USED_CAR_DEEP_LEARNING\\UAE_USED_CAR_DEEP_LEARNING_ANALYSIS_PROJECT'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path   
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list

In [6]:
from USED_CAR_PREDICTION.constants import *
from USED_CAR_PREDICTION.utils.common import read_yaml, create_directories
import tensorflow as tf
import os

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):  
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "car_pic")
        
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        
        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten

# def create_model():
#     model = Sequential([
#         # Add your convolutional layers here
#         Flatten(),
#         Dense(4, activation='softmax')  # Change to 4 for 4 classes
#     ])
#     return model


In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
                
        # # Verify the training data directory path and contents
        # print(f"Training data directory: {self.config.training_data}")
        
        # # List contents in the directory (non-recursive)
        # if Path(self.config.training_data).exists():
        #     files_and_dirs = os.listdir(self.config.training_data)
        #     print(f"Contents of training data directory ({self.config.training_data}): {files_and_dirs}")
        # else:
        #     print(f"Training data directory {self.config.training_data} does not exist.")
            
            
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="validation",
            shuffle=False,
            # target_size=self.config.params_image_size[:-1],
            # batch_size=self.config.params_batch_size,
            # class_mode="categorical",  # Use categorical for multi-class classification
            **dataflow_kwargs
        )
        
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip  = True,
                width_shift_range = 0.2,
                height_shift_range  = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                # rescale= 1./255,
                # validation_split = 0.20,  # Ensure the same split for training
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="training",
            shuffle=True,
            # target_size=self.config.params_image_size[:-1],
            # batch_size=self.config.params_batch_size,
            # class_mode="categorical",  # Use categorical for multi-class classification
            **dataflow_kwargs
        )    
            
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
        
        
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
        

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-06-15 13:06:38,735 : INFO : YAML file config\config.yaml loaded successfully.]
[2025-06-15 13:06:38,738 : INFO : YAML file params.yaml loaded successfully.]
[2025-06-15 13:06:38,739 : INFO : created directory at: artifacts]
[2025-06-15 13:06:38,742 : INFO : created directory at: artifacts\training]
Found 8 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Epoch 1/10
2/2 [==============================] - 3s 2s/step - loss: 83.6089 - accuracy: 0.2917
Epoch 2/10
2/2 [==============================] - 2s 742ms/step - loss: 294.6924 - accuracy: 0.2500
Epoch 3/10
2/2 [==============================] - 3s 1s/step - loss: 157.1985 - accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 2s 736ms/step - loss: 174.1001 - accuracy: 0.3333
Epoch 5/10
2/2 [==============================] - 2s 723ms/step - loss: 241.1248 - accuracy: 0.3333
Epoch 6/10
2/2 [==============================] - 3s 1s/step - loss: 222.7975 - accuracy: 0.2500
Epoch 7/10
2/2 [=========